In [1]:
# import dependencies
import nltk
import json
import io
import gzip
import torch
import string
import random
import jsonlines
import pandas as pd
import pickle as pkl
import numpy as np
from tqdm import tqdm, tqdm_notebook
from functools import partial

import torch
import torch.nn as nn
import torch.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, RandomSampler, SequentialSampler, DataLoader
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
from importlib import reload

import utils
reload(utils)

# import wiki_parser
# reload(wiki_parser)

<module 'utils' from '/home/mz2476/topic-modeling/topic-modeling/baseline/utils.py'>

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [4]:
PATH_TO_DATA_FOLDER = "/scratch/mz2476/wiki/data/aligned_datasets/"

SAVE = True
DEBUG = False
LOAD = False

In [5]:
LANGUAGE = "russian"
LANGUAGE_CODE = LANGUAGE[:2]
FILE_NAMES_DICT = {
    "classes_list": 'classes_list.pt',
    
    "json": f"wikitext_topics_{LANGUAGE_CODE}_filtered.json",
    "wiki_df": f"wikitext_tokenized_text_sections_outlinks_{LANGUAGE_CODE}.p",
#     "vocab": f"vocab_train_{LANGUAGE_CODE}.pt",
#     "train": f"df_wiki_train_{LANGUAGE_CODE}.pt",
#     "val": f"df_wiki_valid_{LANGUAGE_CODE}.pt",
#     "test": f"df_wiki_test_{LANGUAGE_CODE}.pt",
#     "tensor_dataset": f"wiki_tensor_dataset_{LANGUAGE_CODE}.pt",
}

print(FILE_NAMES_DICT)

{'classes_list': 'classes_list.pt', 'json': 'wikitext_topics_ru_filtered.json', 'wiki_df': 'wikitext_tokenized_text_sections_outlinks_ru.p'}


In [6]:
wiki_df_en = pkl.load(open(PATH_TO_DATA_FOLDER + f"wikitext_tokenized_text_sections_outlinks_en.p", "rb"))
wiki_df_ru = pkl.load(open(PATH_TO_DATA_FOLDER + f"wikitext_tokenized_text_sections_outlinks_ru.p", "rb"))
wiki_df_hi = pkl.load(open(PATH_TO_DATA_FOLDER + f"wikitext_tokenized_text_sections_outlinks_hi.p", "rb"))


In [7]:
qids_en = set(wiki_df_en.QID)
qids_ru = set(wiki_df_ru.QID)
qids_hi = set(wiki_df_hi.QID)

In [8]:
# num of unique qid is each language
len(qids_en), len(qids_ru), len(qids_hi)

(33823, 33711, 33637)

In [15]:
print("difference between EN and RU:", len(qids_en.difference(qids_ru)), len(qids_ru.difference(qids_en)))
print("difference between EN and HI:", len(qids_en.difference(qids_hi)), len(qids_hi.difference(qids_en)))
print("difference between RU and HI:", len(qids_ru.difference(qids_hi)), len(qids_hi.difference(qids_ru)))

difference between EN and RU: 112 0
difference between EN and HI: 186 0
difference between RU and HI: 183 109


In [17]:
33823 - 186 - 112 + 3

33528